<a href="https://colab.research.google.com/github/VinhDevNguyen/Project_Alpha/blob/RNN_baseline/RNN_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive
Data for ``Booking Challenge`` stores in ``UIT TSP``folder

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data path

In [34]:
import pandas as pd 

In [35]:
folder_path = './drive/MyDrive/UIT-TSP/Booking.com Data Challenge'
data_path = "/content/drive/MyDrive/UIT-TSP/Booking.com Data Challenge/booking_train_set.csv"

In [36]:
df = pd.read_csv(data_path)
df.head()

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1


# Import libraries

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# RNN with PyTorch 

In [ ]:
class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, 
                 vocab_size, 
                 emb_size=500, 
                 rnn_dim=500, 
                 n_layers=2, 
                 dropout=0.3, 
                 rnn_dropout=0.3, 
                 tie_weights=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout) 
        self.embedding = nn.Embedding(vocab_size,emb_dim) 
        self.rnn = nn.RNN(input_size=emb_dim,
                          hidden_size=rnn_dim, 
                          num_layers=n_layers, 
                          dropout=rnn_dropout) 
        self.dense = nn.Linear(rnn_dim,vocab_size)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights: # theta_d = theta_y
          self.dense.weight = self.embedding.weight

        self.vocab_size = vocab_size 
        self.emb_size = emb_size 
        self.rnn_dim = rnn_dim 
        self.n_layers = n_layers 
        self.dropout = dropout 
        self.rnn_dropout = rnn_dropout 
        self.tie_weights = tie_weights 


    # def init_weights(self):
    #     initrange = 0.1
    #     nn.init.uniform_(self.encoder.weight, -initrange, initrange)
    #     nn.init.zeros_(self.decoder.weight)
    #     nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    # Not clear here 
    def forward(self, input, h0=None): 
      out, hidden = self.rnn(
                      self.drop(
                          self.embedding(input)),
                      h0)
      out = self.dense(self.drop(
                            out.view(
                                out.size(0)*out.size(1),
                                out.size(2)))
                      )
      return out, hidden 

    # def forward(self, input, hidden):
    #     emb = self.drop(self.encoder(input))
    #     output, hidden = self.rnn(emb, hidden)
    #     output = self.drop(output)
    #     decoded = self.decoder(output)
    #     decoded = decoded.view(-1, self.ntoken)
    #     return F.log_softmax(decoded, dim=1), hidden

    # def init_hidden(self, bsz):
    #     weight = next(self.parameters())
    #     if self.rnn_type == 'LSTM':
    #         return (weight.new_zeros(self.nlayers, bsz, self.nhid),
    #                 weight.new_zeros(self.nlayers, bsz, self.nhid))
    #     else:
    #         return weight.new_zeros(self.nlayers, bsz, self.nhid)

    # def save_model(self): 

    # def load_model(self): 



# RNN with Tensorflow 

## Import libraries 

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## create RNN model 

In [ ]:
def create_RNN_model(
          vocab_size, 
          emb_size=500, 
          rnn_dim=500, 
          n_layers=2, 
          dropout=0.3, 
          rnn_dropout=0.3, 
          tie_weights=False):

  model = keras.Sequential()
  model.add(layers.Embedding(input_dim=emb_size, output_dim=4 ))

  # The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
  model.add(layers.GRU(256, return_sequences=True))

  # The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
  model.add(layers.SimpleRNN(128))

  model.add(layers.Dense(10))

  model.summary()

  return model 

In [ ]:
model = create_RNN_model()

## RNN model class 

In [ ]:
class RNNModel:
    def __init__(self, 
                 vocab_size, 
                 emb_size=500, 
                 rnn_dim=500, 
                 n_layers=2, 
                 dropout=0.3, 
                 rnn_dropout=0.3): 
        self.drop = nn.Dropout(dropout) 
        self.embedding = nn.Embedding(vocab_size,emb_dim) 
        self.rnn = nn.RNN(input_size=emb_dim,
                          hidden_size=rnn_dim, 
                          num_layers=n_layers, 
                          dropout=rnn_dropout) 

        self.dense = nn.Linear(rnn_dim,vocab_size)


        self.vocab_size = vocab_size 
        self.emb_size = emb_size 
        self.rnn_dim = rnn_dim 
        self.n_layers = n_layers 
        self.dropout = dropout 
        self.rnn_dropout = rnn_dropout 
        self.tie_weights = tie_weights 

        self.model = create_RNN_model(vocab_size, emb_size, rnn_size, n_layers = n_layers, 
                                      dropout = dropout, rnn_dropout=rnn_dropout,
                                      tie_weights=tie_weights)

    # Not clear here 
    def forward(self, input, h0=None): 
      out, hidden = self.rnn(
                      self.drop(
                          self.embedding(input)),
                      h0)
      out = self.dense(self.drop(
                            out.view(
                                out.size(0)*out.size(1),
                                out.size(2)))
                      )
      return out, hidden 

    # def train(self): 
    # def save_model(self):
    # def load_model(self):

In [2]:
# ## Testing 
# paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
# paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
# paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

# lstm_layer = layers.LSTM(64, stateful=True)
# output = lstm_layer(paragraph1)
# output = lstm_layer(paragraph2)

# existing_state = lstm_layer.states

# new_lstm_layer = layers.LSTM(64)
# new_output = new_lstm_layer(paragraph3, initial_state=existing_state)


In [5]:
# paragraph1 

array([[[0.4384477 , 0.70959634, 0.15308669, ..., 0.28581852,
         0.22975205, 0.3308152 ],
        [0.9046204 , 0.05282678, 0.34636778, ..., 0.104307  ,
         0.95794   , 0.85673577],
        [0.17676865, 0.34466982, 0.46606266, ..., 0.89375216,
         0.90824926, 0.032717  ],
        ...,
        [0.07953037, 0.02336469, 0.9040275 , ..., 0.37990856,
         0.5449542 , 0.11931679],
        [0.13505265, 0.06546313, 0.91656995, ..., 0.96330225,
         0.35127148, 0.30735272],
        [0.43864247, 0.09331185, 0.0787064 , ..., 0.68661904,
         0.21598631, 0.21987194]],

       [[0.5607646 , 0.21337695, 0.24137759, ..., 0.7216155 ,
         0.5272271 , 0.02199812],
        [0.859154  , 0.55872273, 0.8249309 , ..., 0.47798792,
         0.10819671, 0.81250495],
        [0.01605468, 0.9332755 , 0.75010777, ..., 0.02574055,
         0.3090186 , 0.17313221],
        ...,
        [0.4615367 , 0.5239395 , 0.3345207 , ..., 0.31028378,
         0.71332794, 0.7954804 ],
        [0.6

In [3]:
output


<tf.Tensor: shape=(20, 64), dtype=float32, numpy=
array([[-0.26159257,  0.3828493 , -0.06094625, ...,  0.41688082,
         0.06902932,  0.17313975],
       [-0.22398742,  0.3048276 ,  0.01105477, ...,  0.3025591 ,
        -0.09937311,  0.16212305],
       [-0.1873369 ,  0.28067118, -0.0903695 , ...,  0.47735262,
         0.09688166,  0.10506968],
       ...,
       [-0.2334649 ,  0.42825425, -0.08432736, ...,  0.33124694,
         0.13678132,  0.16629355],
       [-0.24908844,  0.33447886, -0.10338344, ...,  0.37441865,
         0.12842214,  0.04302474],
       [-0.12451277,  0.36226118, -0.15007631, ...,  0.34746057,
         0.12647597,  0.1358475 ]], dtype=float32)>

In [4]:
# new_output

<tf.Tensor: shape=(20, 64), dtype=float32, numpy=
array([[ 0.07265107,  0.12564135,  0.2950572 , ..., -0.00305309,
         0.30196252, -0.19266294],
       [ 0.10832786, -0.02208175,  0.19994555, ..., -0.15402594,
         0.15071553, -0.12331202],
       [ 0.04492785, -0.01332777,  0.16630736, ..., -0.03083682,
         0.26744378, -0.19593215],
       ...,
       [ 0.06476085, -0.00676946,  0.22334988, ..., -0.06385501,
         0.20579292, -0.1357603 ],
       [ 0.02144044,  0.03727771,  0.2299651 , ..., -0.06008133,
         0.2050588 , -0.12643524],
       [-0.01233856, -0.00042944,  0.24162439, ..., -0.13264933,
         0.28603524, -0.0941027 ]], dtype=float32)>

In [6]:
# # Test embedding 
# encoder_vocab = 1000
# decoder_vocab = 2000

# encoder_input = layers.Input(shape=(None,))
# encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(
#     encoder_input
# )

# # Return states in addition to output
# output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(
#     encoder_embedded
# )
# encoder_state = [state_h, state_c]

# decoder_input = layers.Input(shape=(None,))
# decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(
#     decoder_input
# )

# # Pass the 2 states to a new LSTM layer, as initial state
# decoder_output = layers.LSTM(64, name="decoder")(
#     decoder_embedded, initial_state=encoder_state
# )
# output = layers.Dense(10)(decoder_output)

# model = keras.Model([encoder_input, decoder_input], output)
# model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     64000       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     128000      input_2[0][0]                    
______________________________________________________________________________________________

# Test Embedding 

In [39]:
df.head()

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1


In [40]:
doc_utrip = df['utrip_id']
print(len(doc_utrip))
doc_utrip.head()

1166835


0    1006220_1
1    1006220_1
2    1006220_1
3    1006220_1
4    1010293_1
Name: utrip_id, dtype: object

In [41]:
# Get label
utrip_label = df['utrip_id'].unique()
print(len(utrip_label))
utrip_label

217686


array(['1006220_1', '1010293_1', '1012680_1', ..., '987787_1', '999261_1',
       '999755_1'], dtype=object)

In [42]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [43]:
vocab_size = 217687
max_length = len(doc_utrip) 
print('vocab size ', vocab_size)
print('max length ', max_length)

vocab size  217687
max length  1166835


In [50]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

model = Sequential()

# Embedding layer
model.add(
   Embedding(1166835,48) 
)
# model.add(
#     Embedding(input_dim=num_words,
#               input_length = training_length,
#               output_dim=100,
#               weights=[embedding_matrix],
#               trainable=False,
#               mask_zero=True))

# # Masking layer for pre-trained embeddings
# model.add(Masking(mask_value=0.0))

# Recurrent layer
model.add(LSTM(64, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 48)          56008080  
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                28928     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 260       
Total params: 56,041,428
Trainable params: 56,041,428
Non-trainable params: 0
_________________________________________________________________


fatal: not a git repository (or any of the parent directories): .git
